In [ ]:
# In[1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %%
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprocess

In [ ]:
import src.slurm_cluster as scluster
client, scluster = scluster.init_dask_slurm_cluster()

In [ ]:
def func_mk(x):
    """
    Mann-Kendall test for trend
    """
    results = data_process.mk_test(x)
    slope = results[0]
    p_val = results[1]
    return slope, p_val

In [ ]:
dir1 ='./Validation/EC_Earth/'
# EC_Earth_forced = xr.open_dataset(dir1 + 'GSAT_EC_Earth_Forced_anomalies_1850_2022.nc')
EC_Earth_internal = xr.open_dataset(dir1 + 'GSAT_EC_Earth_Internal_Variability_anomalies_1850_2022.nc')

### Calculate the trend in 10, 30, 60yr

In [ ]:
variable_name = ['10yr', '30yr', '60yr']
# define the function
def separate_data_into_intervals(data, start_year, end_year, time_interval):
    """
    This function is used to separate the data into different time intervals
    """
    # create a dictionary to store the data
    data_dict = {}
    for i in range(len(variable_name)):
        # calculate the start year and end year
        start_year = time_interval[variable_name[i]][0]
        end_year = time_interval[variable_name[i]][1]
        # select the data
        data_dict[variable_name[i]] = data.sel(year=slice(str(start_year), str(end_year)))
    return data_dict

In [ ]:
time_interval = {
    "10yr":(2013,2022),
    "30yr":(1993,2022),
    "60yr":(1963,2022)
}
start_year = 1950
end_year   = 2022

In [ ]:
EC_Earth_unforced_data_dict = separate_data_into_intervals(EC_Earth_internal, start_year, end_year, time_interval)

In [ ]:
EC_Earth_unforced_data_dict

In [ ]:
# Calculate the trend and p-value for each time interval of each realization
trend_dict = {}
pvalue_dict = {}

for i in range(len(variable_name)):
    data_var = EC_Earth_unforced_data_dict[variable_name[i]]['tas']
    
    slope, p_values = xr.apply_ufunc(
        func_mk,
        data_var,
        input_core_dims=[["year"]],
        output_core_dims=[[], []],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[float, float],
        dask_gufunc_kwargs={'allow_rechunk': True}
    )
    trend_dict[variable_name[i]] = slope
    pvalue_dict[variable_name[i]] = p_values

In [ ]:
trend_dict

In [ ]:
trend_annual_np = {}
pvalue_annual_np = {}

for i in range(len(variable_name)):
    trend_annual_np[variable_name[i]] = trend_dict[variable_name[i]].values
    pvalue_annual_np[variable_name[i]] = pvalue_dict[variable_name[i]].values
    
trend_annual_np['10yr']

In [ ]:
trend_annual_da = {}
pvalue_annual_da = {}

for interval, data in trend_annual_np.items():
    trend_annual_da[interval] = xr.DataArray(data, dims=["run","lat", "lon"], coords={"run": EC_Earth_unforced_data_dict[interval].run, 
    "lat": EC_Earth_unforced_data_dict[interval].lat, "lon": EC_Earth_unforced_data_dict[interval].lon})
for interval, data in pvalue_annual_np.items():
    pvalue_annual_da[interval] = xr.DataArray(data, dims=["run","lat", "lon"], coords={"run": EC_Earth_unforced_data_dict[interval].run, 
    "lat": EC_Earth_unforced_data_dict[interval].lat, "lon": EC_Earth_unforced_data_dict[interval].lon})

In [ ]:
# out put the forced trend and pvalue
dir_out ='./Figure3/EC_Earth/trend/'

for interval, data in trend_annual_da.items():
    data.to_netcdf(dir_out + 'EC_Earth_unforced_' + interval + '_trend.nc')
for interval, data in pvalue_annual_da.items():
    data.to_netcdf(dir_out + 'EC_Earth_unforced_' + interval + '_pvalue.nc')

In [ ]:
trend_annual_da